# SparkContext

In [ ]:
# Spark Hello World

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("WordCount").getOrCreate()

text = "Hello Spark Hello Python Hello Docker Hello World"
words = spark.sparkContext.parallelize(text.split(" "))
wordCounts = words.map(lambda word: (word, 1)). reduceByKey(lambda a, b: a + b)

for wc in wordCounts.collect():
    print(wc[0], wc[1])

spark.stop()

: 

In [ ]:
spark.stop()  # 기존 SparkContext 종료
spark = SparkContext(conf=spark_conf)  # 새로운 SparkContext 시작

In [ ]:
1+1

: 

In [ ]:
from pyspark import SparkConf, SparkContext

spark_conf = SparkConf().setAppName("spark-context-test")
spark = SparkContext.getOrCreate(conf=spark_conf)
print(spark.getConf().getAll())  # 현재 설정 확인

[('spark.executor.id', 'driver'), ('spark.app.name', 'spark-context-test'), ('spark.driver.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true'), ('spark.app.id', 'local-1716997151757'), ('spark.driver.host', '76bea6c276dc'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.driver.port', '37371'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.app.startTime', '1716997151694'), ('spark.executor.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true'), ('spark.ui.showConsoleProgress', 'true')]


# SparkSession

In [22]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .master("local")
        .appName("spark-session-test")
        ## .config("spark.some.config.option", "some-value")
        .getOrCreate()
)
print(f"Hello, {spark.sparkContext.sparkUser()}")

# enableHiveSupport()

Hello, jovyan


# Read Data

In [25]:
df = spark.read.json("./data/2024-01-01-15.json")
df.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

24/05/29 15:42:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [26]:
df.count()

180387

In [27]:
df.show(5)

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+----------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|            type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+----------------+
|{https://avatars....|2024-01-01T15:00:00Z|34502641367|{https://avatars....|{added, null, nul...|  true|{708013309, Uotan...|     MemberEvent|
|{https://avatars....|2024-01-01T15:00:00Z|34502641374|                null|{null, dc8177f8b9...|  true|{732717980, mdmai...|       PushEvent|
|{https://avatars....|2024-01-01T15:00:00Z|34502641375|{https://avatars....|{null, null, null...|  true|{135763852, ample...|       ForkEvent|
|{https://avatars....|2024-01-01T15:00:00Z|34502641376|                null|{closed, null, nu...|  true|{156070344, tobex...|PullRequestEvent|

In [28]:
df = spark.read.format('org.apache.spark.sql.json') \
        .load("./data/2024-01-01-15.json")
df.show(3)

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|       type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------+
|{https://avatars....|2024-01-01T15:00:00Z|34502641367|{https://avatars....|{added, null, nul...|  true|{708013309, Uotan...|MemberEvent|
|{https://avatars....|2024-01-01T15:00:00Z|34502641374|                null|{null, dc8177f8b9...|  true|{732717980, mdmai...|  PushEvent|
|{https://avatars....|2024-01-01T15:00:00Z|34502641375|{https://avatars....|{null, null, null...|  true|{135763852, ample...|  ForkEvent|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------+
only showing top 3 rows



In [29]:
df.dtypes

[('actor',
  'struct<avatar_url:string,display_login:string,gravatar_id:string,id:bigint,login:string,url:string>'),
 ('created_at', 'string'),
 ('id', 'string'),
 ('org',
  'struct<avatar_url:string,gravatar_id:string,id:bigint,login:string,url:string>'),
 ('payload',
  'struct<action:string,before:string,comment:struct<_links:struct<html:struct<href:string>,pull_request:struct<href:string>,self:struct<href:string>>,author_association:string,body:string,commit_id:string,created_at:string,diff_hunk:string,html_url:string,id:bigint,in_reply_to_id:bigint,issue_url:string,line:bigint,node_id:string,original_commit_id:string,original_line:bigint,original_position:bigint,original_start_line:bigint,path:string,performed_via_github_app:struct<created_at:string,description:string,events:array<string>,external_url:string,html_url:string,id:bigint,name:string,node_id:string,owner:struct<avatar_url:string,events_url:string,followers_url:string,following_url:string,gists_url:string,gravatar_id:str

In [30]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, LongType, BooleanType
# https://spark.apache.org/docs/latest/sql-ref-datatypes.html

schema = StructType([
      StructField("actor", StringType(), True),
      StructField("created_at", DateType(), True),
      StructField("id", LongType(), False),
      StructField("org", StringType(), True),
      StructField("payload", StringType(), True),
      StructField("public", BooleanType(), True),
      StructField("repo", StringType(), True),
      StructField("type", StringType(), True)
  ])


In [31]:
df = spark.read.schema(schema).json("./data/2024-01-01-15.json")
df.printSchema()

root
 |-- actor: string (nullable = true)
 |-- created_at: date (nullable = true)
 |-- id: long (nullable = true)
 |-- org: string (nullable = true)
 |-- payload: string (nullable = true)
 |-- public: boolean (nullable = true)
 |-- repo: string (nullable = true)
 |-- type: string (nullable = true)



In [32]:
rdd = sc.textFile('./data/2024-01-01-15.json')
rdd.take(1)

['{"id":"34502641367","type":"MemberEvent","actor":{"id":62330315,"login":"mujianwu","display_login":"mujianwu","gravatar_id":"","url":"https://api.github.com/users/mujianwu","avatar_url":"https://avatars.githubusercontent.com/u/62330315?"},"repo":{"id":708013309,"name":"Uotan-Dev/UotanWorkStation-ROM-Builder","url":"https://api.github.com/repos/Uotan-Dev/UotanWorkStation-ROM-Builder"},"payload":{"member":{"login":"Envoy-Z-Lab","id":152428796,"node_id":"U_kgDOCRXg_A","avatar_url":"https://avatars.githubusercontent.com/u/152428796?v=4","gravatar_id":"","url":"https://api.github.com/users/Envoy-Z-Lab","html_url":"https://github.com/Envoy-Z-Lab","followers_url":"https://api.github.com/users/Envoy-Z-Lab/followers","following_url":"https://api.github.com/users/Envoy-Z-Lab/following{/other_user}","gists_url":"https://api.github.com/users/Envoy-Z-Lab/gists{/gist_id}","starred_url":"https://api.github.com/users/Envoy-Z-Lab/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/use